# Data Scraping

In [1]:
import random 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline 
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs
print('Libraries imported.')

Libraries imported.


In [2]:
import html5lib
from bs4 import BeautifulSoup
import lxml
import sqlalchemy

In [3]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
data1 = data[0]
data1.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Scraping html data, calling first table in webpage

In [4]:
data_trimmed = data1[data1.Borough != 'Not assigned']
data_trimmed.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


Dropping rows with Borough = Not assigned

In [5]:
data2 = data_trimmed.groupby(['Postcode','Borough'], as_index=False)['Neighbourhood'].agg(', '.join)
data3 = pd.DataFrame(data2)
data3.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Grouping Neighbourhoods by Postcode

In [6]:
data3['Neighbourhood'] = np.where(data3['Neighbourhood'] == 'Not assigned', data3['Borough'], data3['Neighbourhood'])
data3

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Replace Not assigned Neighbourhood values with respective Borough value

In [7]:
data3.shape

(103, 3)

In [8]:
import geopy
from geopy.geocoders import Nominatim

In [9]:
!conda install -c conda-forge geocoder --yes
import geocoder


Solving environment: done

# All requested packages already installed.



In [10]:
from IPython.display import clear_output

In [11]:
def get_geocoder(Postcode):
    lat_lng_coords = None

    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(Postcode))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

i=0
lat, lon = [], []
for val in data3['Postcode']:
    lat_value, lon_value = get_geocoder(val)
    lat.append(lat_value)
    lon.append(lon_value)
    clear_output(wait=True)
    i +=1
    print(i/len(data3)*100)
          
data3['Longitude']=lon
data3['Latitude']=lat

100.0


In [12]:
data3.head(25)

,Postcode,Borough,Neighbourhood,Longitude,Latitude
0,M1B,Scarborough,"Rouge, Malvern",-79.195517,43.811525
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",-79.158725,43.785665
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",-79.175193,43.765815
3,M1G,Scarborough,Woburn,-79.217590,43.768369
4,M1H,Scarborough,Cedarbrae,-79.239440,43.769688
5,M1J,Scarborough,Scarborough Village,-79.231750,43.743125
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",-79.263625,43.726276
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",-79.285055,43.713054
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",-79.227925,43.724235
9,M1N,Scarborough,"Birch Cliff, Cliffside West",-79.259967,43.696770


In [13]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: done

# All requested packages already installed.

Folium installed and imported!


In [14]:
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print('Libraries imported.')

Libraries imported.


In [15]:
tor_lat= 43.6532
tor_lon= -79.3832
tor_map = folium.Map(location=[tor_lat, tor_lon], zoom_start=12)
neighbourhoods = folium.map.FeatureGroup()

for lat, lng, in zip(data3.Latitude, data3.Longitude):
    neighbourhoods.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=8, 
            color='blue',
            fill=True,
            fill_color='red',
            fill_opacity=0.8
        )
    )

tor_map.add_child(neighbourhoods)
tor_map

In [16]:
CLIENT_ID = 'FKWRKV1Q3L0QW0ONZCNJ32QLQU3SNRMIHUWW1RIFWQ3WIMW0' # your Foursquare ID
CLIENT_SECRET = 'DCLG1CS014T4YJXE4SUSHTO4SD5FTFBNH3IBXCMHTWF1BA05' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [17]:
LIMIT = 120 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    tor_lat, 
    tor_lon, 
    radius, 
    LIMIT)
results = requests.get(url).json()

In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.shape

(100, 4)

In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'tor_lat', 
                  'tor_lon', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=data3['Neighbourhood'],
                                   latitudes=data3['Latitude'],
                                   longitudes=data3['Longitude']
                                  )



Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West, 

In [25]:
print(toronto_venues.shape)
toronto_venues.head()

(2501, 7)


,Neighbourhood,tor_lat,tor_lon,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.811525,-79.195517,Wood Bison Paddock,43.811732,-79.200708,Zoo Exhibit
1,"Rouge, Malvern",43.811525,-79.195517,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
2,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725,Affordable Toronto Movers,43.787919,-79.162977,Moving Target
3,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
4,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725,Royal Canadian Legion,43.782533,-79.163085,Bar


In [26]:
toronto_venues.groupby('Neighbourhood').count()

,tor_lat,tor_lon,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,17,17,17,17,17,17
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",1,1,1,1,1,1
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",13,13,13,13,13,13
"Alderwood, Long Branch",4,4,4,4,4,4
Bayview Village,5,5,5,5,5,5
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
Berczy Park,62,62,62,62,62,62
"Birch Cliff, Cliffside West",6,6,6,6,6,6


In [27]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,...,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,...,Transportation Service,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,"Adelaide, King, Richmond",0.0,0.0,0.000000,0.00,0.030000,0.000000,0.010000,0.00,0.000000,...,0.000000,0.010000,0.000000,0.000000,0.01000,0.0,0.000000,0.000000,0.000000,0.0
1,Agincourt,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.058824,0.00000,0.0,0.000000,0.000000,0.000000,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0
5,Bayview Village,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0
6,"Bedford Park, Lawrence Manor East",0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0
7,Berczy Park,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.016129,0.00,0.000000,...,0.000000,0.016129,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0
8,"Birch Cliff, Cliffside West",0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0
9,"Bloordale Gardens, Eringate, Markland Wood, Ol...",0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.0


In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.06
2                Hotel  0.05
3           Steakhouse  0.04
4  American Restaurant  0.03


----Agincourt----
                venue  freq
0       Shopping Mall  0.12
1  Chinese Restaurant  0.12
2         Supermarket  0.12
3    Department Store  0.06
4     Bubble Tea Shop  0.06


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                     venue  freq
0                 Pharmacy   1.0
1        Accessories Store   0.0
2      Monument / Landmark   0.0
3  New American Restaurant   0.0
4             Neighborhood   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.15
1       Sandwich Place  0.08
2  Japanese Restaurant  0.08
3         Liquor Store  0.08
4          Coffee Shop  0.08


----Alderwood, Long Branch----
             

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Hotel,Steakhouse,Bar,Japanese Restaurant,Asian Restaurant,Restaurant,Gastropub,American Restaurant
1,Agincourt,Chinese Restaurant,Supermarket,Shopping Mall,Bubble Tea Shop,Department Store,Sushi Restaurant,Bakery,Badminton Court,Discount Store,Shanghai Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Zoo Exhibit,Farm,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Park,Beer Store,Japanese Restaurant,Fried Chicken Joint,Coffee Shop,Liquor Store,Sandwich Place,Hardware Store,Pizza Place
4,"Alderwood, Long Branch",Sandwich Place,Convenience Store,Pub,Gym,Falafel Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Ethiopian Restaurant,Event Space


In [32]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = data3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

 # check the last columns!

In [ ]:
toronto_merged=toronto_merged.loc[toronto_merged["Cluster Labels"].notna()]
toronto_merged.head(25)

In [ ]:
map_clusters = folium.Map(location=[tor_lat, tor_lon], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]